<a href="https://colab.research.google.com/github/geffhubs/pose_voice/blob/main/bert_pose_voice_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 487 kB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#!/usr/bin/env python
# coding: utf-8
#need pytorch==1.5.0
# In[4]:

import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import random
import torch
import torch.nn as nn
from tqdm import tqdm
from scipy.special import softmax

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics


import pandas as pd

import requests
from bs4 import BeautifulSoup

get_ipython().system('pip freeze > requirement.txt')

In [ ]:
def chunkstring(string, length):
    return [string[0+i:length+i] for i in range(0, len(string), length)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import torch

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.

def train_and_validate(model, optimizer, train_dataloader, validation_dataloader, device, weights_filename, train_input_ids, train_attention_masks, train_labels):

  from transformers import get_linear_schedule_with_warmup

  # Number of training epochs. The BERT authors recommend between 2 and 4. 
  # We chose to run for 4, but we'll see later that this may be over-fitting the
  # training data.
  epochs = 4

  # Total number of training steps is [number of batches] x [number of epochs]. 
  # (Note that this is not the same as the number of training samples).
  total_steps = len(train_dataloader) * epochs

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  
  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # The documentation for this `model` function is here: 
          # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
          # It returns different numbers of parameters depending on what arguments
          # arge given and what flags are set. For our useage here, it returns
          # the loss (because we provided labels) and the "logits"--the model
          # outputs prior to activation.
          outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += outputs[0].item() #outputs[0] == loss

          # Perform a backward pass to calculate the gradients.
          outputs[0].backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epoch took: {:}".format(training_time))
      torch.save(model.state_dict(), weights_filename)

    #  print(len(outputs))
     # print(outputs[2])
     # print(outputs.to_tuple())
     # print(outputs['hidden_states'][1:])
      
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
      
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              # The documentation for this `model` function is here: 
              # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
              # Get the "logits" output by the model. The "logits" are the output
              # values prior to applying an activation function like the softmax.
              outputs = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += outputs[0].item()

          # Move logits and labels to CPU
          logits = outputs[1].detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))
      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  return model, training_stats

In [ ]:
def get_data_sentences(books):  

  all=[]
  for book in books:
    text = book[0].read()
    text = preprocess(text)
    label=book[1]
    sentences = sent_tokenize(text)
    for tt in sentences:
        all.append({'text': tt, 'label': label})

  dataset = pd.DataFrame(all)

  train, temp = train_test_split(dataset, random_state=2018, test_size=0.3, stratify=dataset['label'])

  val, test = train_test_split(temp, random_state=2018, test_size=0.5, stratify=temp.label)

  return train, val, test

In [ ]:
from sklearn.metrics import classification_report
from scipy.special import softmax

def print_classification_report(model, test_input_ids, test_attention_masks, test_labels, test, device):
  # get predictions for test data
  with torch.no_grad():
    preds = model(test_input_ids.to(device), test_attention_masks.to(device), labels=test_labels.to(device))
    preds = preds.logits.cpu().numpy()
    float_formatter = "{:.10f}".format
    np.set_printoptions(formatter={'float_kind':float_formatter})
    probabilities = [softmax(pred) for pred in preds]
    preds = np.argmax(preds, axis=1)
  print(classification_report(test.label.values, preds))
  return preds, probabilities


In [ ]:
def get_data(books, train_list=None):
  train=[]
  test=[]
  for book in books:
    text = book[0].read()
    text = preprocess(text)
    label=book[1]
    text_chunks = chunkstring(text, 512)
    for tt in text_chunks:
      if train_list:
        if book[0].name in train_list:
          train.append({'text': tt, 'label': label})
        else:
          test.append({'text': tt, 'label': label})
      else:
          #all books training
           train.append({'text': tt, 'label': label})

  if train_list:
    dataset = pd.DataFrame(train)
    test = pd.DataFrame(test)
    train, val = train_test_split(dataset, random_state=2018, test_size=0.15, stratify=dataset['label'])
    test, leftover = train_test_split(test, random_state=2018, test_size=0.40, stratify=test['label'])
  else:
    dataset=pd.DataFrame(train)
    train, temp = train_test_split(dataset, random_state=2018, test_size=0.3, stratify=dataset['label'])
    val, test = train_test_split(temp, random_state=2018, test_size=0.5, stratify=temp.label)


  return train, val, test

In [ ]:
def get_tokenized(tokenizer, sentences, labels):

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])
  return input_ids, attention_masks, labels

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

def make_dataloaders(train_dataset, val_dataset, test_dataset):
  # The DataLoader needs to know our batch size for training, so we specify it 
  # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
  # size of 16 or 32.
  batch_size = 32

  # Create the DataLoaders for our training and validation sets.
  # We'll take training samples in random order. 
  train_dataloader = DataLoader(
              train_dataset,  # The training samples.
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )

  # For validation the order doesn't matter, so we'll just read them sequentially.
  validation_dataloader = DataLoader(
              val_dataset, # The validation samples.
              sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )

  test_dataloader = DataLoader(
              test_dataset, # The validation samples.
              sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )
  
  return train_dataloader, validation_dataloader, test_dataloader

#model.bert
#model.classifier
#X=model.bert(**inputs)
#X.last_hidden_state[:, 0, :]
#model.classifier.dense(X[0])[:, 0, :]
#CLS --> vector that represents the class

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

def get_model(device, labelnum=3):
  # Load BertForSequenceClassification, the pretrained BERT model with a single 
  # linear classification layer on top. 
  model = BertForSequenceClassification.from_pretrained(
      "bert-base-cased", # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = labelnum, # The number of output labels--2 for binary classification.
                      # You can increase this for multi-class tasks.   
      output_attentions = True, # Whether the model returns attentions weights.
      output_hidden_states = True,
      return_dict=True # Whether the model returns all hidden-states.
  ).to(device)

  # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
  optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

  return model, optimizer

In [ ]:
def print_params(model):
  # Get all of the model's parameters as a list of tuples.
  params = list(model.named_parameters())

  print('The BERT model has {:} different named parameters.\n'.format(len(params)))

  print('==== Embedding Layer ====\n')

  for p in params[0:5]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

  print('\n==== First Transformer ====\n')

  for p in params[5:21]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

  print('\n==== Output Layer ====\n')

  for p in params[-4:]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def load_weights(model, path):
  model.load_state_dict(torch.load(path))
  return model

In [ ]:
def preprocess(text):

  text = text.replace('\n', '').replace("\'", "'").replace("\x0c", " ").replace("\\x0", " ")
  return text

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def run_model(filepaths, weights_file='weights.pt', pose=False, voice = False, train_list=None):
  print("BEGINNING...")
  if pose:
    labs = {'eileen': 0, 'restrelaxation': 0, 'convos': 1, 'normalppl': 1, 'atocha': 2, 'topeka': 2}
    num=3
  elif voice:
    labs = {'beloved': 0, 'bluesteye': 0, 'cathedral': 1, 'willyoubequietplease': 1, 'americanpastoral': 2, 'portnoys_complaint': 2}
    num=3
  else:
    labs = {'eileen': 0, 'restrelaxation': 0, 'convos': 1, 'normalppl': 1, 'atocha': 2, 'topeka': 2, 'beloved': 3, 'bluesteye': 3, 'cathedral': 4, 'willyoubequietplease': 4, 'americanpastoral': 5, 'portnoys_complaint': 5}
    num=6

  books = []
  for path, name in filepaths:
    n = name
    exec("%s = %d" % (name,100))
    name = open(path)
    print(f"Opened {path} as {name}")
    books.append((name, labs[n]))


  train, val, test = get_data(books, train_list)

  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
  train_input_ids, train_attention_masks, train_labels = get_tokenized(tokenizer, list(train.text.values), list(train.label.values))
  test_input_ids, test_attention_masks, test_labels = get_tokenized(tokenizer, list(test.text.values), list(test.label.values))
  val_input_ids, val_attention_masks, val_labels = get_tokenized(tokenizer, list(val.text.values), list(val.label.values)) 
  train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
  test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
  val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
  print("Transformed datasets into tensors....")
  train_dataloader, val_dataloader, test_dataloader = make_dataloaders(train_dataset, val_dataset, test_dataset)
  print("Transformed datasets into dataloaders...")
  model, optimizer = get_model(device=device, labelnum=num)
  print("Retrieved model....")
  print_params(model)
  model, training_stats = train_and_validate(model, optimizer, train_dataloader, val_dataloader, device, weights_file, train_input_ids, train_attention_masks, train_labels)
  preds, probabilities = print_classification_report(model, test_input_ids, test_attention_masks, test_labels, test, device)
  return model, test, preds, probabilities




In [ ]:
filepaths=[('Eileen-Ottessa-Moshfegh.txt', 'eileen'), ('americanpastoral_roth_shortened.txt', 'americanpastoral'), ('beloved-toni-morrison.txt', 'beloved'), ('bluesteye_morrison.txt', 'bluesteye'), ('cathedral_carver.txt', 'cathedral'), ('conversations_with_friends.txt', 'convos'), ('leaving_atocha_station_lerner.txt', 'atocha'), ('my-year-of-rest-and-relaxation.txt', 'restrelaxation'), ('normal_people_rooney.txt', 'normalppl'), ('portnoys_complaint_roth.txt', 'portnoys_complaint'), ('topeka_school.txt', 'topeka'), ('will_you_be_quiet_please_carver.txt', 'willyoubequietplease')]
filepaths_voice=[('americanpastoral_roth_shortened.txt', 'americanpastoral'), ('beloved-toni-morrison.txt', 'beloved'), ('bluesteye_morrison.txt', 'bluesteye'), ('cathedral_carver.txt', 'cathedral'), ('portnoys_complaint_roth.txt', 'portnoys_complaint'), ('will_you_be_quiet_please_carver.txt', 'willyoubequietplease')]
filepaths_pose=[('Eileen-Ottessa-Moshfegh.txt', 'eileen'), ('conversations_with_friends.txt', 'convos'), ('leaving_atocha_station_lerner.txt', 'atocha'), ('my-year-of-rest-and-relaxation.txt', 'restrelaxation'), ('normal_people_rooney.txt', 'normalppl'),  ('topeka_school.txt', 'topeka')]
model, test, preds, probabilities = run_model(filepaths)

BEGINNING...
Opened americanpastoral_roth_shortened.txt as <_io.TextIOWrapper name='americanpastoral_roth_shortened.txt' mode='r' encoding='UTF-8'>
Opened beloved-toni-morrison.txt as <_io.TextIOWrapper name='beloved-toni-morrison.txt' mode='r' encoding='UTF-8'>
Opened bluesteye_morrison.txt as <_io.TextIOWrapper name='bluesteye_morrison.txt' mode='r' encoding='UTF-8'>
Opened cathedral_carver.txt as <_io.TextIOWrapper name='cathedral_carver.txt' mode='r' encoding='UTF-8'>
Opened portnoys_complaint_roth.txt as <_io.TextIOWrapper name='portnoys_complaint_roth.txt' mode='r' encoding='UTF-8'>
Opened will_you_be_quiet_please_carver.txt as <_io.TextIOWrapper name='will_you_be_quiet_please_carver.txt' mode='r' encoding='UTF-8'>


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Original:  , around the time of Kennedy's assassination, beforethe war in Vietnam had begun in earnest, when, as far as everybody knew, Americawas merely at the periphery of whatever was going haywire over there. The monkwho did it was in his seventies, thin, with a shaved head and wearing a saffronrobe. Cross-legged and straight-backed on an empty city street somewhere inSouth Vietnam, gracefully seated like that in front of a crowd of monks who hadgathered to witness the event as though to observe a religious ritual
Token IDs: tensor([  101,   117,  1213,  1103,  1159,  1104,  5107,   112,   188, 10395,
          117,  1196, 10681,  1594,  1107,  4357,  1125,  4972,  1107, 21304,
          117,  1165,   117,  1112,  1677,  1112, 10565,  1450,   117,  1738,
        20963,  5804,  1120,  1103, 27152,  1104,  3451,  1108,  1280, 20433,
        24952,  1166,  1175,   119,  1109, 12112,  2246,  5114,  1225,  1122,
         1108,  1107,  1117,  1978,  4338,   117,  4240,   117,  1114,   17

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Retrieved model....
The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense

RuntimeError: ignored

In [ ]:
res = pd.DataFrame({'text': test.text.values, 'correct': test.label.values, 'predicted': preds, 'probabilities': probabilities})

In [ ]:
res.to_csv('allauthors.csv')

In [ ]:
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/